In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install memory-profiler
!pip install --upgrade psutil
!pip install datasketch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 4.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gzip
import struct
import time
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import scipy
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

In [ ]:
def binarizer(images):
    binary_images = np.where(images > 0, 1, 0).astype('uint8')
    return binary_images

def preprocessing(images, labels=None, n_com=50, type_decomposition='PCA'):
  if type_decomposition=='PCA':
    decomposition = PCA(n_components=n_com)
    decomposition.fit(images)
  else:
    if type_decomposition=='TSNE':
      decomposition = TSNE(n_components=n_com,n_jobs=-1)
      return decomposition
    elif type_decomposition=='LDA':
      decomposition = LDA(n_components=n_com)
    else:
      print("Not a Valid Choice")
      return 0,0
    decomposition.fit(images, labels)
  return decomposition

In [ ]:
class StreamingNaiveBayes:
  def __init__(self, batch_size=100, epsilon=1e-9, alpha=1.0):
    self.batch_size = batch_size
    self.epsilon = epsilon
    self.alpha = alpha
        
  def fit(self, X, y):
    self.classes = np.unique(y)
    self.n_classes = len(self.classes)
    self.n_features = X.shape[1]     
    self.class_counts = np.zeros(self.n_classes, dtype=np.int32)
    self.feature_counts = np.zeros((self.n_classes, self.n_features), dtype=np.float64)
    for i in range(0, X.shape[0], self.batch_size):
      X_batch = X[i:i+self.batch_size]
      y_batch = y[i:i+self.batch_size]      
      self.class_counts += np.bincount(y_batch, minlength=self.n_classes)
      for j in range(self.n_classes):
        mask = (y_batch == self.classes[j])
        count = np.sum(mask)
        self.feature_counts[j] += np.sum(X_batch[mask], axis=0)
    self.feature_probs = (self.feature_counts + self.alpha) / (self.class_counts.reshape(-1, 1) + self.alpha * self.n_features + self.epsilon)
    self.class_probs = self.class_counts / np.sum(self.class_counts)
        
  def predict(self, X):
    log_likelihoods=0
    if np.any((self.feature_probs + self.epsilon).T<=0):
      log_likelihoods = np.dot(X, np.log(np.abs(self.feature_probs + self.epsilon).T)) + np.log(self.class_probs + self.epsilon)
    elif np.any((self.class_probs + self.epsilon)<=0):
      log_likelihoods = np.dot(X, np.log((self.feature_probs + self.epsilon).T)) + np.log(np.abs(self.class_probs + self.epsilon))
    elif np.any((self.class_probs + self.epsilon)<=0) and np.any((self.feature_probs + self.epsilon).T<=0):
      log_likelihoods = np.dot(X, np.log(np.abs(self.feature_probs + self.epsilon).T)) + np.log(np.abs(self.class_probs + self.epsilon))
    else:
      log_likelihoods = np.dot(X, np.log((self.feature_probs + self.epsilon).T)) + np.log(self.class_probs + self.epsilon)
    return self.classes[np.argmax(log_likelihoods, axis=1)]

In [ ]:
with gzip.open('/content/drive/MyDrive/MLBD_Assignment_2/train-images-idx3-ubyte.gz', 'rb') as f:
    _, num_images, num_rows, num_cols = struct.unpack('>IIII', f.read(16))
    images_binary = f.read(num_images*num_rows*num_cols)
    tr_images = np.frombuffer(images_binary, dtype=np.uint8)
    tr_images = tr_images.reshape(num_images, num_rows, num_cols)

with gzip.open('/content/drive/MyDrive/MLBD_Assignment_2/train-labels-idx1-ubyte.gz', 'rb') as f:
    _, num_labels = struct.unpack('>II', f.read(8))
    labels_binary = f.read(num_labels)
    tr_labels = np.frombuffer(labels_binary, dtype=np.uint8)

In [ ]:
with gzip.open('/content/drive/MyDrive/MLBD_Assignment_2/t10k-images-idx3-ubyte.gz', 'rb') as f:
    _, num_images, num_rows, num_cols = struct.unpack('>IIII', f.read(16))
    images_binary = f.read(num_images*num_rows*num_cols)
    te_images = np.frombuffer(images_binary, dtype=np.uint8)
    te_images = te_images.reshape(num_images, num_rows, num_cols)

with gzip.open('/content/drive/MyDrive/MLBD_Assignment_2/t10k-labels-idx1-ubyte.gz', 'rb') as f:
    _, num_labels = struct.unpack('>II', f.read(8))
    labels_binary = f.read(num_labels)
    te_labels = np.frombuffer(labels_binary, dtype=np.uint8)

In [ ]:
tr_images = tr_images.reshape(tr_images.shape[0],-1)
te_images = te_images.reshape(te_images.shape[0],-1)
print(tr_images.shape,te_images.shape)

(60000, 784) (10000, 784)


In [ ]:
tr_images, te_images = binarizer(tr_images), binarizer(te_images)

#Without applying Dimensionality Reduction

## Stream_Window=10, Epsilon=1e-9, Alpha=1.0

In [ ]:
nb = StreamingNaiveBayes(batch_size=10, epsilon=1e-9, alpha=1.0)
start_tr = time.time()
nb.fit(tr_images, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_images)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 1.4578919410705566 seconds
Accuracy: 62.43%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.05203413963317871 seconds



## Stream_Window=100, Epsilon=1e-9, Alpha=1.0

In [ ]:
nb = StreamingNaiveBayes(batch_size=100, epsilon=1e-9, alpha=1.0)
start_tr = time.time()
nb.fit(tr_images, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_images)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.260390043258667 seconds
Accuracy: 62.43%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.04279661178588867 seconds



## Stream_Window=10, Epsilon=1, Alpha=1.0

In [ ]:
nb = StreamingNaiveBayes(batch_size=10, epsilon=1, alpha=1.0)
start_tr = time.time()
nb.fit(tr_images, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_images)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 1.5151591300964355 seconds
Accuracy: 54.96%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.04021000862121582 seconds



## Stream_Window=100, Epsilon=1, Alpha=1.0

In [ ]:
nb = StreamingNaiveBayes(batch_size=100, epsilon=1, alpha=1.0)
start_tr = time.time()
nb.fit(tr_images, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_images)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.22924590110778809 seconds
Accuracy: 54.96%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.04111123085021973 seconds



## Stream_Window=10, Epsilon=1e9, Alpha=1.0

In [ ]:
nb = StreamingNaiveBayes(batch_size=10, epsilon=1e9, alpha=1.0)
start_tr = time.time()
nb.fit(tr_images, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_images)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 1.9037749767303467 seconds
Accuracy: 11.35%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.06326532363891602 seconds



## Stream_Window=100, Epsilon=1e9, Alpha=1.0

In [ ]:
nb = StreamingNaiveBayes(batch_size=100, epsilon=1e9, alpha=1.0)
start_tr = time.time()
nb.fit(tr_images, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_images)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.37812066078186035 seconds
Accuracy: 11.35%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.06345319747924805 seconds



## Stream_Window=10, Epsilon=1e-9, Alpha=10.0

In [ ]:
nb = StreamingNaiveBayes(batch_size=10, epsilon=1e-9, alpha=10.0)
start_tr = time.time()
nb.fit(tr_images, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_images)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 2.687964677810669 seconds
Accuracy: 63.43%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.0432126522064209 seconds



## Stream_Window=10, Epsilon=1e-9, Alpha=0.1

In [ ]:
nb = StreamingNaiveBayes(batch_size=10, epsilon=1e-9, alpha=0.1)
start_tr = time.time()
nb.fit(tr_images, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_images)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 1.3595151901245117 seconds
Accuracy: 62.35000000000001%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.04393148422241211 seconds



## Stream_Window=10, Epsilon=1e-9, Alpha=100.0

In [ ]:
nb = StreamingNaiveBayes(batch_size=10, epsilon=1e-9, alpha=100.0)
start_tr = time.time()
nb.fit(tr_images, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_images)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 1.9494762420654297 seconds
Accuracy: 64.25%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.06657075881958008 seconds



## Stream_Window=10, Epsilon=1e-9, Alpha=0.01

In [ ]:
nb = StreamingNaiveBayes(batch_size=10, epsilon=1e-9, alpha=0.01)
start_tr = time.time()
nb.fit(tr_images, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_images)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 2.9556729793548584 seconds
Accuracy: 62.36000000000001%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.07645153999328613 seconds



#While using Dimensionality Reduction

##PCA

###n_components=50

In [ ]:
mode = preprocessing(tr_images, tr_labels, 50, 'PCA')
tr_image, te_image = mode.transform(tr_images), mode.transform(te_images)
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.5567362308502197 seconds
Accuracy: 11.99%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.021024703979492188 seconds



In [ ]:
mode = preprocessing(tr_images, tr_labels, 50, 'PCA')
images = mode.fit_transform(np.vstack((tr_images, te_images)))
tr_image, te_image = images[:60001,:], images[60000:,:]
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.281444787979126 seconds
Accuracy: 11.92%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.005118370056152344 seconds



###n_components=100

In [ ]:
mode = preprocessing(tr_images, tr_labels, 100, 'PCA')
tr_image, te_image = mode.transform(tr_images), mode.transform(te_images)
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.28940558433532715 seconds
Accuracy: 12.25%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.006029844284057617 seconds



In [ ]:
mode = preprocessing(tr_images, tr_labels, 100, 'PCA')
images = mode.fit_transform(np.vstack((tr_images, te_images)))
tr_image, te_image = images[:60001,:], images[60000:,:]
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.5494375228881836 seconds
Accuracy: 12.379999999999999%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.022900819778442383 seconds



###n_components=200

In [ ]:
mode = preprocessing(tr_images, tr_labels, 200, 'PCA')
tr_image, te_image = mode.transform(tr_images), mode.transform(te_images)
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.30446887016296387 seconds
Accuracy: 12.22%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.010184049606323242 seconds



In [ ]:
mode = preprocessing(tr_images, tr_labels, 200, 'PCA')
images = mode.fit_transform(np.vstack((tr_images, te_images)))
tr_image, te_image = images[:60001,:], images[60000:,:]
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.13631772994995117 seconds
Accuracy: 12.41%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.013323307037353516 seconds



##LDA

###n_components=2

In [ ]:
mode = preprocessing(tr_images, tr_labels, 2, 'LDA')
tr_image, te_image = mode.transform(tr_images), mode.transform(te_images)
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.16003680229187012 seconds
Accuracy: 8.61%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.0014553070068359375 seconds



In [ ]:
mode = preprocessing(tr_images, tr_labels, 2, 'LDA')
img = np.vstack((tr_images, te_images))
lab = np.vstack((np.expand_dims(tr_labels,1), np.expand_dims(te_labels,1)))
images = mode.fit_transform(img, lab)
tr_image, te_image = images[:60001,:], images[60000:,:]
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Time taken to fit Stream Naive Bayes model: 0.16337156295776367 seconds
Accuracy: 8.450000000000001%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.0020051002502441406 seconds



###n_components=5

In [ ]:
mode = preprocessing(tr_images, tr_labels, 5, 'LDA')
tr_image, te_image = mode.transform(tr_images), mode.transform(te_images)
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.15996742248535156 seconds
Accuracy: 3.74%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.003038167953491211 seconds



In [ ]:
mode = preprocessing(tr_images, tr_labels, 5, 'LDA')
img = np.vstack((tr_images, te_images))
lab = np.vstack((np.expand_dims(tr_labels,1), np.expand_dims(te_labels,1)))
images = mode.fit_transform(img, lab)
tr_image, te_image = images[:60001,:], images[60000:,:]
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Time taken to fit Stream Naive Bayes model: 0.15761017799377441 seconds
Accuracy: 4.6899999999999995%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.0023555755615234375 seconds



###n_components=8

In [ ]:
mode = preprocessing(tr_images, tr_labels, 8, 'LDA')
tr_image, te_image = mode.transform(tr_images), mode.transform(te_images)
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.15912151336669922 seconds
Accuracy: 8.68%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.004147768020629883 seconds



In [ ]:
mode = preprocessing(tr_images, tr_labels, 8, 'LDA')
img = np.vstack((tr_images, te_images))
lab = np.vstack((np.expand_dims(tr_labels,1), np.expand_dims(te_labels,1)))
images = mode.fit_transform(img, lab)
tr_image, te_image = images[:60001,:], images[60000:,:]
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Time taken to fit Stream Naive Bayes model: 0.15784263610839844 seconds
Accuracy: 10.23%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.0035169124603271484 seconds



##TSNE

###n_components=1

In [ ]:
mode = preprocessing(tr_images, tr_labels, 1, 'TSNE')
tr_image, te_image = mode.fit_transform(tr_images), mode.fit_transform(te_images)
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.11501264572143555 seconds
Accuracy: 17.349999999999998%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.0018575191497802734 seconds



In [ ]:
mode = preprocessing(tr_images, tr_labels, 1, 'TSNE')
images = mode.fit_transform(np.vstack((tr_images, te_images)))
tr_image, te_image = images[:60001,:], images[60000:,:]
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.19522833824157715 seconds
Accuracy: 18.82%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.002174854278564453 seconds



###n_components=2

In [ ]:
mode = preprocessing(tr_images, tr_labels, 2, 'TSNE')
tr_image, te_image = mode.fit_transform(tr_images), mode.fit_transform(te_images)
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.1160731315612793 seconds
Accuracy: 10.09%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.002921581268310547 seconds



In [ ]:
mode = preprocessing(tr_images, tr_labels, 2, 'TSNE')
images = mode.fit_transform(np.vstack((tr_images, te_images)))
tr_image, te_image = images[:60001,:], images[60000:,:]
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.13177132606506348 seconds
Accuracy: 3.2199999999999998%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.003055095672607422 seconds



###n_components=3

In [ ]:
mode = preprocessing(tr_images, tr_labels, 3, 'TSNE')
tr_image, te_image = mode.fit_transform(tr_images), mode.fit_transform(te_images)
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.11559057235717773 seconds
Accuracy: 7.71%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.004822254180908203 seconds



In [ ]:
mode = preprocessing(tr_images, tr_labels, 3, 'TSNE')
images = mode.fit_transform(np.vstack((tr_images, te_images)))
tr_image, te_image = images[:60001,:], images[60000:,:]
nb = StreamingNaiveBayes()
start_tr = time.time()
nb.fit(tr_image, tr_labels)
end_tr = time.time()
fit_tr = end_tr - start_tr
print("Time taken to fit Stream Naive Bayes model:", fit_tr, "seconds")
start_te = time.time()
y_pred = nb.predict(te_image)
accuracy = np.sum(y_pred == te_labels) / len(te_labels)
print(f"Accuracy: {accuracy*100}%")
end_te = time.time()
fit_te = end_te - start_te
print("Time taken to evaluate Stream Naive Bayes model on test portion:", fit_te, "seconds\n")

Time taken to fit Stream Naive Bayes model: 0.15100622177124023 seconds
Accuracy: 14.92%
Time taken to evaluate Stream Naive Bayes model on test portion: 0.003951549530029297 seconds

